In [1]:
import numpy as np
import pandas as pd

def func_f(y, lmda):
    out = np.abs(y) - lmda
    out = out * (out >= 0)
    return np.sum(out)

def find_lambda(y, c):
    list_lmd_0, list_lmd_1 = [], []
    for i in range(len(y)):
        tmp = func_f(y, np.abs(y[i]))
        if tmp >= c:
            list_lmd_0.append(np.abs(y[i]))
        if tmp <= c:
            list_lmd_1.append(np.abs(y[i]))
    lmda_1 = min(list_lmd_1)
    if len(list_lmd_0) != 0:
        lmda_0 = max(list_lmd_0)
    else:
        lmda_0 = 0
    if lmda_0 == lmda_1:
        return lmda_0
    return lmda_0 + ((c - func_f(y, lmda_0))/(func_f(y, lmda_0) - func_f(y, lmda_1))) * (lmda_0 - lmda_1)

def project(y, c):
    lmda = find_lambda(y, c)
    x = np.abs(y) - lmda
    x = x * (x >= 0)
    x = np.sign(y) * x 
    return x

def problem2_2():
    y = pd.read_csv('homework4_data/project4_vector-1.csv')
    y = y.drop(['Unnamed: 0'], axis = 1).to_numpy()
    y = np.squeeze(y)
    list_c = [5, 10, 20, 100]
    for c in list_c:
        out = np.linalg.norm(y - project(y, c))**2
        print("The value of " + r'$|y-proj_{\Omega_c}(y)|^2$' + " for c = " + str(c) + " is: " + str(out))

In [2]:
if __name__ == "__main__":
    problem2_2()

The value of $|y-proj_{\Omega_c}(y)|^2$ for c = 5 is: 9797.346376053942
The value of $|y-proj_{\Omega_c}(y)|^2$ for c = 10 is: 9768.217407278022
The value of $|y-proj_{\Omega_c}(y)|^2$ for c = 20 is: 9713.883606540363
The value of $|y-proj_{\Omega_c}(y)|^2$ for c = 100 is: 9340.632835889344


In [3]:
def conditional_grad(y, c):
    def find_j(x, y):
        j, max_j = 0, 0
        for i in range(len(y)):
            tmp = np.abs(x[i]-y[i])
            if max_j < tmp:
                max_j = tmp
                j=i
        return j

    x = np.zeros(len(y))
    for t in range(10000):
        alpha = 2/(2+t)
        j = find_j(x, y)
        e = np.zeros(len(y))
        e[j] = 1
        x = (1-alpha)*x-c*alpha*np.sign(x[j]-y[j])*e
    return x

def problem2_5():
    y = pd.read_csv('homework4_data/project4_vector-1.csv')
    y = y.drop(['Unnamed: 0'], axis = 1).to_numpy()
    y = np.squeeze(y)
    list_c = [5, 10, 20, 100]
    for c in list_c:
        x_T = conditional_grad(y, c)
        print("The value of " + r'$|y-x_T|^2$' + " for c = " + str(c) + " is: " 
              + str(np.linalg.norm(y - x_T)**2))
        print("The value of " + r'$|x_T-proj_{\Omega_c}(y)|^2$' + " for c = " + str(c) + " is: " 
              + str(np.linalg.norm(x_T - project(y, c))**2))

In [4]:
if __name__ == "__main__":
    problem2_5()

The value of $|y-x_T|^2$ for c = 5 is: 9797.346377942433
The value of $|x_T-proj_{\Omega_c}(y)|^2$ for c = 5 is: 1.8884932352928527e-06
The value of $|y-x_T|^2$ for c = 10 is: 9768.217424264963
The value of $|x_T-proj_{\Omega_c}(y)|^2$ for c = 10 is: 1.2463814546991488e-05
The value of $|y-x_T|^2$ for c = 20 is: 9713.883736900732
The value of $|x_T-proj_{\Omega_c}(y)|^2$ for c = 20 is: 0.00010089560776718761
The value of $|y-x_T|^2$ for c = 100 is: 9340.645931864432
The value of $|x_T-proj_{\Omega_c}(y)|^2$ for c = 100 is: 0.010320629431984777


In [5]:
def conditional_grad_2(A, b, c):
    def find_j(u):
        j, max_j = 0, 0
        for i in range(len(u)):
            tmp = np.abs(u[i])
            if max_j < tmp:
                max_j = tmp
                j = i
        return j

    x = np.zeros(A.shape[1])
    for t in range(100000):
        u = np.dot(A.T, np.dot(A,x)-b)
        alpha = 2/(2+t)
        j = find_j(u)
        e = np.zeros(A.shape[1])
        e[j] = 1
        x = (1-alpha)*x-c*alpha*np.sign(u[j])*e
    return x

def problem2_7():
    A = pd.read_csv('homework4_data/project4_regression_A-1.csv')
    A = A.drop(['Unnamed: 0'], axis = 1).to_numpy()
    A = np.squeeze(A)
    b = pd.read_csv('homework4_data/project4_regression_b.csv')
    b = b.drop(['Unnamed: 0'], axis = 1).to_numpy()
    b = np.squeeze(b)
    list_c = [5, 10, 15]
    for c in list_c:
        x_T = conditional_grad_2(A, b, c)
        print("The value of " + r'$|Ax_T-b|^2$' + " for c = " + str(c) + " is: " 
              + str(np.linalg.norm(np.dot(A,x_T)-b)**2))
        list_max_x_T = np.sort(x_T)[-c:]
        print("The indexes of the c = " + str(c) + " largest number in the set " 
              + r'$\{x_T^{(k)},k=1,\cdots,n\}$' + " is: ")
        for k in range(A.shape[1]):
            if x_T[k] in list_max_x_T:
                print(k, end = " ")
        print()

In [6]:
if __name__ == "__main__":
    problem2_7()

The value of $|Ax_T-b|^2$ for c = 5 is: 1361.2936729535782
The indexes of the c = 5 largest number in the set $\{x_T^{(k)},k=1,\cdots,n\}$ is: 
104 362 372 619 767 
The value of $|Ax_T-b|^2$ for c = 10 is: 42.69954220008175
The indexes of the c = 10 largest number in the set $\{x_T^{(k)},k=1,\cdots,n\}$ is: 
104 362 372 451 477 538 619 629 767 789 
The value of $|Ax_T-b|^2$ for c = 15 is: 0.002448971584146125
The indexes of the c = 15 largest number in the set $\{x_T^{(k)},k=1,\cdots,n\}$ is: 
104 217 362 372 451 477 520 523 538 619 629 767 789 863 990 


In [7]:
def projected_GD(A, b, c, alpha):
    x = np.zeros(A.shape[1])
    for t in range(100000):
        grad = np.dot(A.T, np.dot(A,x)-b)
        x = project(x-alpha*grad, c)
    return x

def problem2_8():
    A = pd.read_csv('homework4_data/project4_regression_A-1.csv')
    A = A.drop(['Unnamed: 0'], axis = 1).to_numpy()
    A = np.squeeze(A)
    b = pd.read_csv('homework4_data/project4_regression_b.csv')
    b = b.drop(['Unnamed: 0'], axis = 1).to_numpy()
    b = np.squeeze(b)
    list_c = [5, 10, 15]
    list_alpha = [1e-3, 1e-3, 1e-4]
    for i in range(len(list_c)):
        c = list_c[i]
        alpha = list_alpha[i]
        x_T = projected_GD(A, b, c, alpha)
        print("The value of " + r'$|Ax_T-b|^2$' + " for c = " + str(c) + " is: " 
              + str(np.linalg.norm(np.dot(A,x_T)-b)**2))
        list_max_x_T = np.sort(x_T)[-c:]
        print("The indexes of the c = " + str(c) + " largest number in the set " 
              + r'$\{x_T^{(k)},k=1,\cdots,n\}$' + " is: ")
        for k in range(A.shape[1]):
            if x_T[k] in list_max_x_T:
                print(k, end = " ")
        print()

In [8]:
if __name__ == "__main__":
    problem2_8()

The value of $|Ax_T-b|^2$ for c = 5 is: 1361.2936662674565
The indexes of the c = 5 largest number in the set $\{x_T^{(k)},k=1,\cdots,n\}$ is: 
104 362 372 619 767 
The value of $|Ax_T-b|^2$ for c = 10 is: 42.699437957778756
The indexes of the c = 10 largest number in the set $\{x_T^{(k)},k=1,\cdots,n\}$ is: 
104 362 372 451 477 538 619 629 767 789 


<ipython-input-1-af43e24f1bee>:24: RuntimeWarning: invalid value encountered in double_scalars
  return lmda_0 + ((c - func_f(y, lmda_0))/(func_f(y, lmda_0) - func_f(y, lmda_1))) * (lmda_0 - lmda_1)


ValueError: ignored